In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BooleanType
import json
import requests


In [0]:
storage_account_name = "testemarnes"
container_name = "output"
storage_account_key = "Wcq8QGGLwDDOj1wcSYvL5vDXC4ZWqBGl6QdAACmicZ0ldoj+huGDc9SayZicCU2WGW1YiKFHBV4T+AStjHvKzA=="

In [0]:
try:
    # Check if the mount point already exists
    dbutils.fs.ls("/mnt/your_mount_point")
except Exception as e:
    # If it doesn't exist, mount it
    if "No such file or directory" in str(e):
        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
            mount_point="/mnt/your_mount_point",
            extra_configs={"fs.azure.account.key." + storage_account_name + ".blob.core.windows.net": storage_account_key}
        )
    else:
        # Handle other exceptions
        print(f"Error: {str(e)}")


In [0]:
dbutils.fs.ls("/mnt/your_mount_point")

[FileInfo(path='dbfs:/mnt/your_mount_point/dataparquet/', name='dataparquet/', size=0, modificationTime=1706675651000),
 FileInfo(path='dbfs:/mnt/your_mount_point/output.txt', name='output.txt', size=201478, modificationTime=1706671775000),
 FileInfo(path='dbfs:/mnt/your_mount_point/output_Meli.txt', name='output_Meli.txt', size=171536, modificationTime=1709949049000)]

In [0]:
url_source = "dbfs:/mnt/your_mount_point/output_Meli.txt"

In [0]:
df = spark.read.json(url_source)
df.show()

+--------------------+--------------------+-------------------------+--------------------+-----------------+--------------------+----------+--------------------+-------+--------------------+
|   available_filters|     available_sorts|country_default_time_zone|             filters|           paging|        pdp_tracking|     query|             results|site_id|                sort|
+--------------------+--------------------+-------------------------+--------------------+-----------------+--------------------+----------+--------------------+-------+--------------------+
|[{official_store,...|[{price_asc, Meno...|                GMT-03:00|[{category, Categ...|{50, 0, 577, 577}|{false, [{MLA2123...|chromecast|[{true, [{OTHERS,...|    MLA|{relevance, Más r...|
+--------------------+--------------------+-------------------------+--------------------+-----------------+--------------------+----------+--------------------+-------+--------------------+



In [0]:
row = df.select('results.id').collect()[0]['id']

In [0]:
dados_a_Analyzar = []
for id in row:
    url = f"https://api.mercadolibre.com/items/{id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Process the data further (e.g., convert to dataframe)
        dados_a_Analyzar.append([
            data["id"],
            data["site_id"],
            data["title"],
            data["seller_id"],
            data["category_id"],
            data["official_store_id"],
            float(data["price"]) if data["price"] is not None else 0.0,
            float(data["base_price"]) if data["base_price"] is not None else 0.0,
            float(data["original_price"]) if data["original_price"] is not None else 0.0,
            data["currency_id"],
            data["initial_quantity"],
            data["buying_mode"],
            data["listing_type_id"],
            data["condition"]
        ])
    else:
        print(f"Error: API call failed with status code: {response.status_code}")


In [0]:
schema_final = StructType([
    StructField("id", StringType(), True),
    StructField("site_id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("seller_id", IntegerType(), True),
    StructField("category_id", StringType(), True),
    StructField("official_store_id", IntegerType(), True),
    StructField("price", FloatType(), True),
    StructField("base_price", FloatType(), True),
    StructField("original_price", FloatType(), True),
    StructField("currency_id", StringType(), True),
    StructField("initial_quantity", StringType(), True),
    StructField("buying_mode", StringType(), True),
    StructField("listing_type_id", StringType(), True),
    StructField("condition", StringType(), True)
])

In [0]:
df_final = spark.createDataFrame(dados_a_Analyzar, schema=schema_final)

In [0]:
df_final.show()

+-------------+-------+--------------------+----------+-----------+-----------------+--------+----------+--------------+-----------+----------------+-----------+---------------+---------+
|           id|site_id|               title| seller_id|category_id|official_store_id|   price|base_price|original_price|currency_id|initial_quantity|buying_mode|listing_type_id|condition|
+-------------+-------+--------------------+----------+-----------+-----------------+--------+----------+--------------+-----------+----------------+-----------+---------------+---------+
|MLA1375500305|    MLA|Chromecast Google...| 154567859|  MLA352001|             NULL| 67500.0|   67500.0|           0.0|        ARS|             270| buy_it_now|   gold_special|      new|
|MLA1520576020|    MLA|Chromecast Google...|1473286957|  MLA352001|             NULL| 71790.0|   71790.0|           0.0|        ARS|             390| buy_it_now|   gold_special|      new|
|MLA1696276176|    MLA|Google Chromecast...| 216203805|  MLA

In [0]:
# Substitua 'caminho/para/salvar/no/datalake' pelo caminho desejado no seu Data Lake
# Specify the directory path in the Data Lake
directory_path = "dbfs:/mnt/your_mount_point/"

# List the files in the directory
files_list = dbutils.fs.ls(directory_path)

# Count the number of files
number_of_files = len(files_list)


camino_no_datalake = f"dbfs:/mnt/your_mount_point/file_csv_{number_of_files}.csv"
# Escrever o DataFrame no Data Lake
df_final.write.csv(camino_no_datalake, header=True)

In [0]:
df_final.show()

+-------------+-------+--------------------+----------+-----------+-----------------+--------+----------+--------------+-----------+----------------+-----------+---------------+---------+
|           id|site_id|               title| seller_id|category_id|official_store_id|   price|base_price|original_price|currency_id|initial_quantity|buying_mode|listing_type_id|condition|
+-------------+-------+--------------------+----------+-----------+-----------------+--------+----------+--------------+-----------+----------------+-----------+---------------+---------+
|MLA1375500305|    MLA|Chromecast Google...| 154567859|  MLA352001|             NULL| 67500.0|   67500.0|           0.0|        ARS|             270| buy_it_now|   gold_special|      new|
|MLA1520576020|    MLA|Chromecast Google...|1473286957|  MLA352001|             NULL| 71790.0|   71790.0|           0.0|        ARS|             390| buy_it_now|   gold_special|      new|
|MLA1696276176|    MLA|Google Chromecast...| 216203805|  MLA